In [1]:
import os
import sys
import torchvision
from transformers.tokenization_bert import BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I1125 10:13:32.902130 140581297116992 file_utils.py:39] PyTorch version 1.1.0 available.
I1125 10:13:33.234193 140581297116992 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [100]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from transformers.tokenization_bert import BertTokenizer
import config
from fgc_preprocess import FgcSerDataset, BertIdx
from torch.utils.data import DataLoader
from sup_model import BertSupSentClassification
from transformers import BertModel
from tqdm import tqdm_notebook as tqdm
import torch
import torch.nn as nn
import torchvision
from transformers.tokenization_bert import BertTokenizer
import config
from fgc_preprocess import FgcSerDataset, BertIdx
from torch.utils.data import DataLoader
from sup_model import BertSupSentClassification
from transformers import BertModel
from transformers.optimization import AdamW, WarmupLinearSchedule
from tqdm import tqdm
import ujson
import json
import config
import numpy as np
import torch
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import Dataset
from stanfordcorenlp import StanfordCoreNLP

In [15]:
nlp = StanfordCoreNLP('http://140.109.19.191', port=9000, lang='zh')
props = {'annotators': 'ssplit', 'ssplit.boundaryTokenRegex': '[。]|[!?！？]+',
         'outputFormat': 'json', 'pipelineLanguage': 'zh', 'timeout': '5000000'}

def sentence_split(dtext):
    anno = json.loads(nlp.annotate(dtext))

    out_sents = []
    for s in anno['sentences']:
        s_start = s['tokens'][0]['characterOffsetBegin']
        s_end = s['tokens'][-1]['characterOffsetEnd']
        s_string = dtext[s_start: s_end]

        out_sents.append(s_string)

    return out_sents

I1125 10:25:55.222210 140581297116992 corenlp.py:42] Using an existing server http://140.109.19.191:9000
I1125 10:25:56.226137 140581297116992 corenlp.py:118] The server is available.


In [33]:
def convert2ids(tokenizer, sentences, question):
    input_ids_all = torch.zeros((len(sentences), 512), dtype=torch.long)
    token_type_ids_all = torch.zeros((len(sentences), 512), dtype=torch.long)
    attention_mask_all = torch.zeros((len(sentences), 512), dtype=torch.long)
    
    for s_i, sentence in enumerate(sentences):
        input_ids = torch.zeros(512, dtype=torch.long)
        token_type_ids = torch.zeros(512, dtype=torch.long)
        attention_mask = torch.zeros(512, dtype=torch.long)

        tokenized_q = ['[CLS]'] + tokenizer.tokenize(question) + ['[SEP]']
        tokenized_all = tokenized_q + tokenizer.tokenize(sentence)

        if len(tokenized_all) > 511:
            print("tokenized all > 511")
            tokenized_all = tokenized_all[:512]
        tokenized_all += ['[SEP]']
        ids_all = tokenizer.convert_tokens_to_ids(tokenized_all)

        input_ids[:len(ids_all)] = torch.tensor(ids_all)
        token_type_ids[len(tokenized_q):len(tokenized_all)] = 1
        attention_mask[:len(tokenized_all)] = 1
        
        input_ids_all[s_i] = input_ids
        token_type_ids_all[s_i] = token_type_ids
        attention_mask_all[s_i] = attention_mask

    return input_ids_all, token_type_ids_all, attention_mask_all

In [37]:
class SER_extract:
    def __init__(self):
        bert_model_name = 'bert-base-chinese'
        bert_encoder = BertModel.from_pretrained(bert_model_name)
        model = BertSupSentClassification(bert_encoder)
        model.load_state_dict(torch.load('trainedmodel.m'))
        model.eval()
        
        self.tokenizer = tokenizer = BertTokenizer.from_pretrained(bert_model_name)
        self.model = model
        
    def predict(self, context_sents, question):
        input_ids, token_type_ids, attention_mask = convert2ids(self.tokenizer, context_sents, question)
        
        with torch.no_grad():
            logits = self.model(input_ids, token_type_ids=token_type_ids,
						 attention_mask=attention_mask, mode=BertSupSentClassification.ForwardMode.EVAL)
        return logits

In [38]:
ser_extracter = SER_extract()

I1125 10:30:00.518281 140581297116992 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
I1125 10:30:00.520668 140581297116992 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "fi

In [48]:
with open(config.FGC_TRAIN, 'r') as f:
    items = ujson.load(f)

In [49]:
len(items)

51

In [ ]:
def get_f()

In [ ]:
from fgc_preprocess import prepro_all
def eval():
    items = prepro_all(config.FGC_TRAIN)
    
    tp = 0
    gol_t = 0
    gol_t = 0
    
    
    for item in items:
        context_sents = sentence_split(item['DTEXT'])
        for question in item['QUESTIONS']:
            logits = ser_extracter.predict(context_sents, question)
            prediction = torch.sigmoid(logits) > 0.5
            prediction = prediction.squeeze(1).numpy()
            gold = np.array(question['SUP_EVIDENCE'])
            
            gol_t = np.count_nonzero(gold == 1)
            pre_t =  np.count_nonzero(prediction == 1)
            
            for i, gs in enumerate(g):
                if gs == t[i] == 1:
                    tp += 1
            
    print('prection = {}'.format(tp / pre_t))
    
            

In [80]:
t = prediction.squeeze(1).numpy()

In [ ]:
tp = 0
gol = 0
pre = 0

gol_t = np.count_nonzero(np.array(g) == 1)
pre_t =  np.count_nonzero(t) == 1)

for i, gs in enumerate(g):
    if gs == t[i] == 1:
        tp += 1
recall = tp / gol_t
precision = tp / pre_t